In [1]:
import pandas as pd
import numpy as np
import time

## School Progress Report (SPR) Data
Only need enrollment data from here

In [2]:
# read SPR data
spr = pd.read_excel('data/SPR_SY1617_Data/SPR_SY1617_School_Metric_Scores_20180206.xlsx', sheet_name=1)
# spr = pd.read_excel('data/', sheet_name=1)

# convert School ID to string
spr = spr.rename(columns={'SRC School ID': 'School ID'})
spr['School ID'] = spr['School ID'].astype(str)

school_info = spr[['School ID', 'School', 'Enrollment']].drop_duplicates()

school_info.head(3)

,School ID,School,Enrollment
0,101,John Bartram High School,601
1,102,West Philadelphia High School,483
2,103,High School of the Future,476


## Serious Incidents (aka Safety)

In [3]:
# pd.read_csv("/Users/battix/Desktop/w209-data-viz/philly-schools/data/")
si_1314 = pd.read_csv('data/Serious_Incidents/Serious_Incidents_2013-14.TXT')
si_1415 = pd.read_excel('data/Serious_Incidents/Serious_Incidents_2014-15.xlsx', sheet_name=0)
si_1415.rename(columns={'ULCS Code':'ULCS_NO', 
                        'School Year':'SCHOOL_YEAR', 
                        'Incident Type':'INCIDENT_TYPE',
                        'Incident Count':'INCIDENT_COUNT',
                        'School ID':'SCHOOL_ID'}, inplace=True)

si_1516 = pd.read_excel('data/Serious_Incidents/Serious_Incidents_2015-16.xls', sheet_name=0)
si_1617 = pd.read_excel('data/Serious_Incidents/Serious_Incidents_2016-17.xlsx', sheet_name=0)

si = pd.concat([si_1314, si_1415, si_1516, si_1617]).reset_index(drop=True)
si.drop('ULCS_NO', axis=1, inplace=True)
si['INCIDENT_TYPE'] = si['INCIDENT_TYPE'].str.upper()
# si['INCIDENT_TYPE'].replace('SUICIDE - ATTEMPTS & THREATS', 'SUICIDE - ATTEMPTS AND THREATS', inplace=True)
# si['INCIDENT_TYPE'].replace('DRUGS & ALCOHOL OFFENSE', 'DRUG AND ALCOHOL OFFENSES', inplace=True)
# si['INCIDENT_TYPE'].replace('DRUG & ALCOHOL OFFENSES', 'DRUG AND ALCOHOL OFFENSES', inplace=True)
si['INCIDENT_TYPE'].replace('THREATS', 'THREAT', inplace=True)
si['INCIDENT_TYPE'].replace('THEFTS', 'THEFT', inplace=True)
# si['INCIDENT_TYPE'].replace('BOMB SCARES', 'BOMB SCARE', inplace=True)
# si['INCIDENT_TYPE'].replace('FIRES AND FALSE ALARMS', 'FIRE - INCENDIARY', inplace=True)
# si['INCIDENT_TYPE'].replace('THREATS - VERBAL AND WRITTEN', 'THREAT', inplace=True)
si['INCIDENT_TYPE'].replace('WEAPONS OFFENSES', 'WEAPONS', inplace=True)
si.head()

,INCIDENT_COUNT,INCIDENT_TYPE,SCHOOL_ID,SCHOOL_YEAR
0,1,HARASSMENT,526,2013-2014
1,5,ASSAULTS,534,2013-2014
2,8,THREAT,535,2013-2014
3,0,TAKE OFF,537,2013-2014
4,0,DRUGS & ALCOHOL OFFENSE,545,2013-2014


In [4]:
# convert School ID to string
si['SCHOOL_ID'] = si['SCHOOL_ID'].astype(str)

df = pd.merge(si, school_info, how='left', left_on='SCHOOL_ID', right_on='School ID')

incidents_to_keep = ['ASSAULTS', 'BULLYING', 'BURGLARY', 'ROBBERY', 'THEFT', 'HARASSMENT', 'SUICIDE', 'WEAPONS']
df = df[df['INCIDENT_TYPE'].isin(incidents_to_keep)]

df.drop('School ID', axis=1, inplace=True)
df.head()

,INCIDENT_COUNT,INCIDENT_TYPE,SCHOOL_ID,SCHOOL_YEAR,School,Enrollment
0,1,HARASSMENT,526,2013-2014,Lewis Elkin School,866.0
1,5,ASSAULTS,534,2013-2014,James R. Ludlow School,312.0
5,0,BURGLARY,548,2013-2014,General Philip Kearny School,401.0
7,0,BULLYING,552,2013-2014,Kensington High School for Creative and Perfor...,505.0
8,0,WEAPONS,553,2013-2014,Philip H. Sheridan School,730.0


In [5]:
df_4years = df.groupby(['INCIDENT_TYPE', 'SCHOOL_ID', 'School']).sum()
df_4years = df_4years.reset_index()
df_4years.head()

,INCIDENT_TYPE,SCHOOL_ID,School,INCIDENT_COUNT,Enrollment
0,ASSAULTS,101,John Bartram High School,71,2404.0
1,ASSAULTS,102,West Philadelphia High School,40,1932.0
2,ASSAULTS,103,High School of the Future,52,1904.0
3,ASSAULTS,105,Paul Robeson High School for Human Services,1,891.0
4,ASSAULTS,110,William L. Sayre High School,70,1968.0


In [6]:
df_4years['Incident_rate'] = (df_4years['INCIDENT_COUNT'] * 100) / (df_4years['Enrollment'] * 4)

df_4years['INCIDENT_TYPE'] = df_4years['INCIDENT_TYPE'].apply(lambda x: x.title())
df_4years = df_4years[~pd.isnull(df_4years['SCHOOL_ID'])]

df_4years['TotalIncidentRate'] = df_4years['Incident_rate'].groupby(df_4years['SCHOOL_ID']).transform('sum')

df_4years.head()

,INCIDENT_TYPE,SCHOOL_ID,School,INCIDENT_COUNT,Enrollment,Incident_rate,TotalIncidentRate
0,Assaults,101,John Bartram High School,71,2404.0,0.738353,1.528702
1,Assaults,102,West Philadelphia High School,40,1932.0,0.517598,1.125776
2,Assaults,103,High School of the Future,52,1904.0,0.682773,1.413690
3,Assaults,105,Paul Robeson High School for Human Services,1,891.0,0.028058,0.140292
4,Assaults,110,William L. Sayre High School,70,1968.0,0.889228,1.452405


# Export Data

In [7]:
# save data
# df_4years.to_csv("data/philly_school_safety.csv", sep='|', index=False)

In [8]:
df_4years[df_4years['School'].str.contains('Treaty')]

,INCIDENT_TYPE,SCHOOL_ID,School,INCIDENT_COUNT,Enrollment,Incident_rate,TotalIncidentRate
89,Assaults,516,Penn Treaty High School,180,2248.0,2.001779,3.314057
299,Bullying,516,Penn Treaty High School,0,1686.0,0.000000,3.314057
508,Burglary,516,Penn Treaty High School,0,1686.0,0.000000,3.314057
717,Harassment,516,Penn Treaty High School,20,2248.0,0.222420,3.314057
926,Robbery,516,Penn Treaty High School,18,2248.0,0.200178,3.314057
1134,Suicide,516,Penn Treaty High School,4,1124.0,0.088968,3.314057
1343,Theft,516,Penn Treaty High School,14,2248.0,0.155694,3.314057
1553,Weapons,516,Penn Treaty High School,58,2248.0,0.645018,3.314057


In [9]:
# df_4years.head()